# Reich


This demo runs two copies of a single composer generator to produce a very short version of Steve Reich's Piano Phase. 

----

Imports from the musx package, see [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for more information:

In [ ]:
from musx import Note, Seq, Score, MidiFile, Cycle, keynum, playfile, setmidiplayer
print('OK!')

If you have installed a terminal-based midi player you can autoplay this notebook's generated midi file(s) by deleting the *#* comment sign below and providing your  terminal play command without including any file name. The unmodified example would call [fluidsynth](https://www.fluidsynth.org/download/) and pass it an instance of the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) soundfont stored in /usr/local/sf/:

In [ ]:
# setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf2")

The piano_phase() composer generates musical events to an output score:

In [ ]:
def piano_phase(score, end, keys, rate):
    """
    Composes a piano part for Steve Reich's Piano Phase.

    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    end : int | float
        The total duration of the piece.
    keys : list
        A list of midi key numbers to play in a loop.
    rate : int | float
        The rhythm to use.    
    """
    # Create a cyclic pattern to produce the key numbers.
    pattern = Cycle(keys)
    # Generate notes until score time is >= end.
    while score.now < end:
        # Get the next key number.
        knum = pattern.read()
        # Create a midi note to play it.
        note = Note(time=score.now, duration=rate, pitch=knum, amplitude=.9)
        # Add the midi note to the score.
        score.add(note)
        # Return the amount of time until this composer runs again.
        yield rate
print(f"piano_phase: {piano_phase}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack()
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Convert Reich's piano notes into a list of midi key numbers:

In [ ]:
keys = keynum("e4 f# b c#5 d f#4 e c#5 b4 f# d5 c#")
print(f"keys: {keys}")

Create two instances of the piano_phase() composer and run them at slightly different rates to cause Reich's phasing effect:

In [ ]:
pianos = [piano_phase(score, 20, keys, .167), 
          piano_phase(score, 20, keys, .170)]
print(f"pianos: {pianos}")

Compose the composition:

In [ ]:
score.compose(pianos)
print(f"score: {score}")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("reich.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if you activated setmidiplayer() at the top of the file:

In [ ]:
playfile(file.pathname)